In [9]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
from pbg import PBG
from util import Loader
from sklearn.pipeline import Pipeline
from preprocessor import Preprocessor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

# carrega matrix
s_dataset = '/exp/datasets/docs_rotulados/classic3'
l = Loader()
d = l.from_files(s_dataset)

print(len(d['corpus']))
# Préprocessamento
for i,s in enumerate(d['corpus']):
    if len(s.split()) < 10:
        del d['class_index'][i]
        del d['corpus'][i]
print(len(d['corpus']))

cvect = CountVectorizer()
steps = [('preprocessor',Preprocessor()), ('countvectorizer',cvect)]
pipe = Pipeline(steps) 
pipe.fit(d['corpus'])
M = pipe.transform(d['corpus'])


7095
6175


In [50]:
from collections import defaultdict

def init_vectors(M,y,k, unlabelled_idx=-1, beta=0.0):
    ndocs,nwords = M.shape
    B = np.full((nwords,k),beta)
    count={}
    for word in range(nwords): count[word] = defaultdict(int)
    rows,cols = M.nonzero()
    for row,col in zip(rows,cols):
        label = y[row]
        if label != unlabelled_idx:
            count[col][label] += M[row,col]
            count[col][-1] += M[row,col]
    for word in range(nwords):
        for cls in count[word]:
            if cls != -1: B[word][cls] = (beta + count[word][cls])/(beta + count[word][-1])
    return B

M2 = np.array([[1,2,4,0,4],[1,2,0,10,4],[2,4,0,2,4]])
y2 = np.array([0,1,-1])

init_vectors(M2,y2,2)

array([[0.5, 0.5],
       [0.5, 0.5],
       [1. , 0. ],
       [0. , 1. ],
       [0.5, 0.5]])

In [10]:
from util import ConfigLabels
%reload_ext autoreload
%autoreload 2

# escolhes quantos dados rotulados
number_labelled_examples = [10,20,30,40,50,100] # 10,20,..,50 exemplos rotulados por classe
conf_labels = ConfigLabels(list_n_labels=number_labelled_examples)
conf_labels.fit(d['class_index']) 

In [22]:

from pbg import PBG
from util import RandMatrices

%reload_ext autoreload
%autoreload 2

# teste com 50 dados rotulados
n_examples_labelled = 10
#vetor y com os rótulos, apenas 50 exemplos rotulados
y = conf_labels.semi_labels[n_examples_labelled]
# numero de classes
K = len(np.unique(d['class_index'])) 

pbg = PBG(K, alpha=0.05, beta=0.001, local_max_itr=10, 
              global_max_itr=10, local_threshold = 1e-6, global_threshold = 1e-6, 
              max_time=18000, save_interval=-1, out_dir='.', out_A='A', out_B='B', calc_q=False, debug=False)
# executa pbg transdutivo

pbg

PBG(alpha=0.05, beta=0.001, calc_q=False, debug=False, global_max_itr=10,
  global_threshold=1e-06, local_max_itr=10, local_threshold=1e-06,
  max_time=18000, n_components=4, out_A='A', out_B='B', out_dir='.',
  rand_init=False, save_interval=-1)

In [23]:
pbg.fit(M, y)

PBG(alpha=0.05, beta=0.001, calc_q=False, debug=False, global_max_itr=10,
  global_threshold=1e-06, local_max_itr=10, local_threshold=1e-06,
  max_time=18000, n_components=4, out_A='A', out_B='B', out_dir='.',
  rand_init=False, save_interval=-1)

In [24]:
from sklearn.metrics import classification_report, confusion_matrix

y_unlabelled = conf_labels.unlabelled_idx[n_examples_labelled]
predicted_labels = pbg.transduction_[y_unlabelled]
true_labels = np.array(d['class_index'])[y_unlabelled]

cm = confusion_matrix(true_labels, predicted_labels, labels=pbg.classes_)
print(classification_report(true_labels, predicted_labels))
print("Confusion matrix")
print(cm)

             precision    recall  f1-score   support

          0       0.95      0.99      0.97      1388
          1       0.84      0.94      0.89      1450
          2       0.95      0.87      0.91      2274
          3       1.00      0.97      0.99      1023

avg / total       0.93      0.93      0.93      6135

Confusion matrix
[[1372    6   10    0]
 [   2 1361   86    1]
 [  64  235 1973    2]
 [   4   21    2  996]]


In [35]:
#ABAIXO ESTÁ EXEMPLO COM O LABEL PROPAGATION

In [15]:
from sklearn.semi_supervised import label_propagation
from sklearn.metrics import classification_report, confusion_matrix
from scipy.sparse import csgraph

# Learn with LabelSpreading
label_spread = label_propagation.LabelSpreading(kernel='knn', alpha=0.8)
label_spread.fit(M.toarray(), y)

/home/thiagodepaulo/anaconda3/lib/python3.6/site-packages/sklearn/semi_supervised/label_propagation.py:288: ConvergenceWarning: max_iter=30 was reached without convergence.
  category=ConvergenceWarning


LabelSpreading(alpha=0.8, gamma=20, kernel='knn', max_iter=30, n_jobs=1,
        n_neighbors=7, tol=0.001)

In [16]:
y_unlabelled = conf_labels.unlabelled_idx[n_examples_labelled]
predicted_labels = label_spread.transduction_[y_unlabelled]
true_labels = np.array(d['class_index'])[y_unlabelled]

cm = confusion_matrix(true_labels, predicted_labels, labels=label_spread.classes_)
print(classification_report(true_labels, predicted_labels))
print("Confusion matrix")
print(cm)

             precision    recall  f1-score   support

          0       0.67      0.25      0.36        57
          1       0.10      0.93      0.18        14
          2       1.00      0.20      0.33         5
          3       0.73      0.12      0.21        90

avg / total       0.67      0.23      0.26       166

Confusion matrix
[[14 39  0  4]
 [ 1 13  0  0]
 [ 0  4  1  0]
 [ 6 73  0 11]]
